In [1]:
from tvDatafeed import TvDatafeed, Interval
import mysql.connector
import pandas as pd

In [2]:
def query(query, host="ideatrade.serveftp.net", port=51410, user="Wasan@sun", password=")Ai_C40bj]6ssI9W", database="db_ideatrade"):
    with mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            columns = [column[0] for column in cursor.description]
            result = cursor.fetchall()

    return pd.DataFrame(columns=columns, data=result)

In [3]:
to_database_vm = query("SELECT symbol, exchange FROM allsymbol_global;")
to_database_vm.to_csv("to_database_vm.csv", index=False)

In [4]:
# ! pip install gspread google-auth google-auth-oauthlib google-auth-httplib2
# ! pip install --upgrade certifi
# ! pip install mysql-connector-python
# ! pip install google-auth

import pandas as pd
pd.options.display.float_format = "{:,.3f}".format

import numpy as np
import datetime
import mysql.connector
import yfinance as yf
import requests

import warnings
warnings.filterwarnings('ignore')
from functools import lru_cache

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'tahoma'
import matplotlib.ticker as ticker

import gspread
from google.oauth2.service_account import Credentials
#import gspread_dataframe as gd

In [5]:
from datetime import datetime, timedelta
def get_data():
    # โหลดข้อมูลรับรอง JSON จากไฟล์ที่คุณดาวน์โหลด
    credentials = Credentials.from_service_account_file('keys.json', scopes=['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])

    # ตั้งค่าการเชื่อมต่อ gspread ด้วยข้อมูลรับรอง
    gc = gspread.service_account(filename='keys.json')

    # เปิดสเปรดชีตของ Google Sheets โดยใช้ชื่อหรือ URL
    spreadsheet = gc.open('stores_shopee')
 
    # เลือกชีตที่ต้องการใน Google Sheets
    worksheet = spreadsheet.worksheet('ชีต2')

    # อ่านข้อมูลจากชีตเป็นรายการของดิกชันนารี
    data = worksheet.get_all_records()

    # สร้าง DataFrame จากข้อมูล
    df = pd.DataFrame(data)

    return df
df = get_data()

In [6]:

vm = 1
df_vm = df[df['vm'] == vm]
df_vm.reset_index(drop=True, inplace=True)


In [7]:
df_vm

,symbol,exchange,vm
0,ZZZ,NEO,1
1,ZZG,BER,1
2,ZYUS,ASX,1
3,ZYME,NASDAQ,1
4,ZYDUSLIFE,BSE,1
...,...,...,...
26460,23,SZSE,1
26461,19,SZSE,1
26462,12,SZSE,1
26463,8,SZSE,1


In [8]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
from tqdm import tqdm

tv = TvDatafeed()

def get_historical_data(tv, symbol, exchange):
    try:
        # Convert numeric symbol to string
        if isinstance(symbol, (int, float)):
            symbol = str(symbol)

        # Check if symbol and exchange are strings before using strip()
        if isinstance(symbol, str) and isinstance(exchange, str):
            symbol = symbol.strip()
            exchange = exchange.strip()

            intervals = [
                Interval.in_1_minute, Interval.in_3_minute, Interval.in_5_minute,
                Interval.in_15_minute, Interval.in_30_minute, Interval.in_45_minute,
                Interval.in_1_hour, Interval.in_2_hour, Interval.in_3_hour,
                Interval.in_4_hour, Interval.in_daily, Interval.in_weekly, Interval.in_monthly
            ]

            data = [tv.get_hist(symbol=symbol, exchange=exchange, interval=interval,
                                n_bars=10000 if 'daily' in str(interval) else 6000) for interval in intervals]

            return pd.concat(data)

        else:
            print(f"Skipping {symbol} on {exchange}: Symbol and exchange must be strings.")
            return pd.DataFrame()  # Return an empty DataFrame for symbols that couldn't be retrieved
    except Exception as e:
        print(f"Error retrieving data for {symbol} on {exchange}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame for the symbol that couldn't be retrieved

# Execute the query to get symbols and exchanges
sym_ex = df_vm[['symbol', 'exchange']]

# Iterate through symbols and exchanges, and fetch data for each combination
all_data = []
s = 0
for ind, row in tqdm(sym_ex.iterrows(), "Getting TV"):
    symbol = row['symbol']
    exchange = row['exchange']
    data = get_historical_data(tv, symbol, exchange)
    s += 1
    if not data.empty:
        all_data.append(data)
    if s == 3:
        break

# Concatenate the data for all symbols and exchanges
result = pd.concat(all_data)

# Now, 'result' contains the concatenated historical data for all symbols in every exchange
# You can proceed to use or store this data as needed


you are using nologin method, data you access may be limited
Getting TV: 0it [00:00, ?it/s]

Getting TV: 2it [01:48, 54.30s/it]


In [9]:

# result[result['symbol'] == 'NEO:ZZZ']

In [10]:
print(result[result['symbol'] == 'NEO:ZZZ'])

                      symbol   open   high    low  close      volume
datetime                                                            
2022-07-11 20:44:00  NEO:ZZZ 24.820 24.820 24.820 24.820     100.000
2022-07-11 22:23:00  NEO:ZZZ 24.740 24.740 24.740 24.740     100.000
2022-07-11 22:28:00  NEO:ZZZ 24.720 24.720 24.720 24.720     200.000
2022-07-11 22:48:00  NEO:ZZZ 24.750 24.750 24.750 24.750     100.000
2022-07-11 23:32:00  NEO:ZZZ 24.830 24.830 24.830 24.830     100.000
...                      ...    ...    ...    ...    ...         ...
2023-08-01 20:30:00  NEO:ZZZ 28.790 28.800 23.350 24.700 276,197.000
2023-09-01 20:30:00  NEO:ZZZ 24.180 24.180 22.210 23.270 103,220.000
2023-10-02 20:30:00  NEO:ZZZ 23.190 23.190 21.360 21.860 178,866.000
2023-11-01 20:30:00  NEO:ZZZ 21.960 24.490 21.600 23.910  62,165.000
2023-12-01 21:30:00  NEO:ZZZ 24.040 26.590 24.000 25.800  42,325.000

[53029 rows x 6 columns]


In [11]:
df_symbol = df_vm[['symbol']].drop_duplicates()
df_exchage = df_vm[['exchange']].drop_duplicates()


In [12]:

count_int = 0
count_str = 0
count_t = 0
lst = []

df_exchage = df_vm[['exchange']].drop_duplicates()

for i in df_symbol['symbol']:
    if type(i) == int:
        count_int += 1
    elif type(i) == str:
        
        count_str += 1
    else :
        count_t += 1
        print(type(i))
        lst.append(i)
# count_int, count_str, count_t
# lst


<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


In [13]:
count_int, count_str, count_t

(4855, 19492, 4)

In [14]:
lst[0]

inf

In [15]:

result_symbol = result[['symbol']].drop_duplicates()
result_symbol

,symbol
datetime,
2022-07-11 20:44:00,NEO:ZZZ
2009-02-02 14:40:00,BER:ZZG
2021-12-20 06:04:00,ASX:ZYUS


In [16]:
result_symbol.count()

symbol    3
dtype: int64

In [17]:
# index
nifty_index_data = tv.get_hist(symbol='NIFTY',exchange='NSE',interval=Interval.in_1_minute,n_bars=10000)

# futures continuous contract
nifty_futures_data = tv.get_hist(symbol='NIFTY',exchange='NSE',interval=Interval.in_1_hour,n_bars=1000,fut_contract=1)

# crudeoil
crudeoil_data = tv.get_hist(symbol='CRUDEOIL',exchange='MCX',interval=Interval.in_1_hour,n_bars=5000,fut_contract=1)

# downloading data for extended market hours
extended_price_data = tv.get_hist(symbol="EICHERMOT",exchange="NSE",interval=Interval.in_1_hour,n_bars=500, extended_session=False)


ERROR:tvDatafeed.main:The read operation timed out
ERROR:tvDatafeed.main:no data, please check the exchange and symbol


In [18]:
nifty_futures_data

,symbol,open,high,low,close,volume
datetime,,,,,,
2023-05-31 12:45:00,NSE:NIFTY1!,"18,624.400","18,637.450","18,572.000","18,577.500","981,700.000"
2023-05-31 13:45:00,NSE:NIFTY1!,"18,577.250","18,604.000","18,566.000","18,586.050","716,100.000"
2023-05-31 14:45:00,NSE:NIFTY1!,"18,585.050","18,623.900","18,582.300","18,600.600","561,400.000"
2023-05-31 15:45:00,NSE:NIFTY1!,"18,600.900","18,651.550","18,594.500","18,622.200","1,091,350.000"
2023-05-31 16:45:00,NSE:NIFTY1!,"18,622.050","18,641.950","18,620.700","18,624.950","528,750.000"
...,...,...,...,...,...,...
2023-12-27 12:45:00,NSE:NIFTY1!,"21,592.300","21,619.950","21,578.000","21,601.000","596,400.000"
2023-12-27 13:45:00,NSE:NIFTY1!,"21,601.000","21,602.100","21,530.100","21,554.000","1,520,850.000"
2023-12-27 14:45:00,NSE:NIFTY1!,"21,552.600","21,589.400","21,526.000","21,578.000","919,150.000"


In [19]:
nifty_index_data

,symbol,open,high,low,close,volume
datetime,,,,,,
2023-12-04 10:45:00,NSE:NIFTY,"20,602.500","20,602.500","20,537.200","20,543.000","13,331,122.000"
2023-12-04 10:46:00,NSE:NIFTY,"20,544.950","20,557.600","20,539.700","20,555.100","5,756,084.000"
2023-12-04 10:47:00,NSE:NIFTY,"20,553.600","20,559.000","20,547.450","20,547.450","3,445,582.000"
2023-12-04 10:48:00,NSE:NIFTY,"20,549.600","20,550.950","20,531.100","20,534.200","2,847,064.000"
2023-12-04 10:49:00,NSE:NIFTY,"20,532.200","20,541.550","20,528.900","20,530.550","4,538,630.000"
...,...,...,...,...,...,...
2023-12-27 16:55:00,NSE:NIFTY,"21,666.250","21,667.600","21,663.300","21,664.350","1,041,050.000"
2023-12-27 16:56:00,NSE:NIFTY,"21,665.900","21,671.150","21,665.300","21,670.850","1,143,282.000"
2023-12-27 16:57:00,NSE:NIFTY,"21,669.750","21,675.750","21,669.750","21,672.100","1,140,529.000"
